<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Biomass Results

_by Jeremy Freeman (CarbonPlan), October 25, 2020_

This notebook extracts and summarizes results from fitting biomass growth
curves. See the notebook `biomass_modeling` for how the model is fit. This
notebook assumes the model has been run and results have been saved using the
script `biomass.py`.


In [1]:
import pickle
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load

In [2]:
setup.plotting(remote=False)

First we load our model results. This assumes the model has been run and results
have been saved using the script `biomass.py`.


In [3]:
pf = pd.read_parquet(
    "/Users/freeman/github/carbonplan/forests/scripts/data/biomass.parquet"
)

Filter out bad values


In [4]:
badinds = np.isnan(pf["historical"])
for key in pf.columns:
    if key not in ["lat", "lon", "type_code", "r2"]:
        badinds = badinds | np.isnan(pf[key])
pf = pf[~badinds]

We can plot the results from the model, evaluting on the same climate as used
for fitting


In [5]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["historical"],
    cmap="yellowgreen",
    clim=(0, 250),
    size=1,
)

alt.LayerChart(...)

We can also use other columns to look at future scenarios. For example, here's
the difference in biomass from 2020 to 2100.


In [6]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["BCC-CSM2-MR_ssp245_2100"] - pf["BCC-CSM2-MR_ssp245_2020"],
    cmap="yellowgreen",
    clim=(0, 100),
    size=1,
)

alt.LayerChart(...)

Make a time series of spatially averaged projections


In [7]:
years = np.arange(2020, 2120, 20)
y1 = [pf["BCC-CSM2-MR_ssp245_" + str(year)].mean() for year in years]
y2 = [pf["BCC-CSM2-MR_ssp370_" + str(year)].mean() for year in years]
y3 = [pf["BCC-CSM2-MR_ssp585_" + str(year)].mean() for year in years]
opts = {"color": "rgb(75,128,97)", "ylabel": "biomass", "xlabel": "year"}
(
    plot.line(x=years, y=y1, opacity=0.5, ylim=[100, 175], **opts)
    + plot.line(x=years, y=y2, opacity=0.75, **opts)
    + plot.line(x=years, y=y3, opacity=1, **opts)
)

alt.LayerChart(...)